# COVID - 19

Questions:



Datasource: Google Cloud - Public Datasets, Covid19_usafacts

In [1]:
import pandas as pd
import numpy as np


In [2]:
#create dataframe 
cases = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\confirmed_cases.csv')
deaths = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\deaths.csv')


In [3]:
#manipulate data: remove columns & unpivot
cases_drop = cases.drop(columns=['county_fips_code','state_fips_code'])
cases_unpivot = cases_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_cases')
cases_unpivot['date'] = cases_unpivot.date.str[1:].replace('_', '-',regex=True)
deaths_drop = deaths.drop(columns=['county_fips_code','state_fips_code'])
deaths_unpivot = deaths_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_deaths')
deaths_unpivot['date'] = deaths_unpivot.date.str[1:].replace('_', '-',regex=True)

deaths_unpivot['date'] = pd.to_datetime(deaths_unpivot['date'])
deaths_unpivot = deaths_unpivot.sort_values(['state','county_name'])
cases_unpivot['date'] = pd.to_datetime(cases_unpivot['date'])
cases_unpivot = cases_unpivot.sort_values(['state','county_name'])

#remove rolling deaths
##previous row comparison
county_match_d = deaths_unpivot['county_name'] == deaths_unpivot['county_name'].shift(1) 
state_match_d = deaths_unpivot['state'] == deaths_unpivot['state'].shift(1)
date_match_d = deaths_unpivot['date'] == deaths_unpivot['date'].shift(1) + pd.DateOffset(1)
rolling_deaths_1_d = deaths_unpivot['rolling_deaths'].shift(1) == 0
rolling_deaths_2_d = deaths_unpivot['rolling_deaths'] > deaths_unpivot['rolling_deaths'].shift(1) 
rolling_deaths_3_d = deaths_unpivot['rolling_deaths'] == deaths_unpivot['rolling_deaths'].shift(1) 
true_1_d = deaths_unpivot['rolling_deaths'] - deaths_unpivot['rolling_deaths'].shift(1)

deaths_unpivot['deaths'] = np.where(
                                    (county_match_d & 
                                    state_match_d & 
                                    date_match_d) &
                                    (rolling_deaths_1_d | rolling_deaths_2_d | rolling_deaths_3_d),
                                    true_1_d,
                                    0)


#remove rolling cases
##previous row comparison
county_match_c = cases_unpivot['county_name'] == cases_unpivot['county_name'].shift(1) 
state_match_c = cases_unpivot['state'] == cases_unpivot['state'].shift(1)
date_match_c = cases_unpivot['date'] == cases_unpivot['date'].shift(1) + pd.DateOffset(1)
rolling_deaths_1_c = cases_unpivot['rolling_cases'].shift(1) == 0
rolling_deaths_2_c = cases_unpivot['rolling_cases'] > cases_unpivot['rolling_cases'].shift(1) 
rolling_deaths_3_c = cases_unpivot['rolling_cases'] == cases_unpivot['rolling_cases'].shift(1) 
true_1_c = cases_unpivot['rolling_cases'] - cases_unpivot['rolling_cases'].shift(1)

cases_unpivot['confirmed_cases'] = np.where(
                                    (county_match_c & 
                                    state_match_c & 
                                    date_match_c) &
                                    (rolling_deaths_1_c | rolling_deaths_2_c | rolling_deaths_3_c),
                                    true_1_c,
                                    0)






In [18]:
#merge dataframes
df = pd.merge(cases_unpivot,deaths_unpivot)
df = df.drop(columns=['rolling_cases','rolling_deaths'])
df


,county_name,state,date,confirmed_cases,deaths
0,Municipality of Anchorage,AK,2020-01-22,0.0,0.0
1,Municipality of Anchorage,AK,2020-01-23,0.0,0.0
2,Municipality of Anchorage,AK,2020-01-24,0.0,0.0
3,Municipality of Anchorage,AK,2020-01-25,0.0,0.0
4,Municipality of Anchorage,AK,2020-01-26,0.0,0.0
...,...,...,...,...,...
390973,Natrona County,WY,2021-09-17,35.0,0.0
390974,Natrona County,WY,2021-09-18,0.0,0.0
390975,Natrona County,WY,2021-09-19,0.0,0.0
390976,Natrona County,WY,2021-09-20,0.0,0.0


## Which top 5 states has the most covid cases?

In [10]:
df.groupby('state').sum().sort_values('confirmed_cases', ascending=False).head(5).round({'confirmed_cases': 0}).astype(int).reset_index()

,state,confirmed_cases,deaths
0,CA,4123619,64019
1,FL,2957593,30802
2,TX,2957235,40293
3,NY,2233921,52790
4,IL,1212623,20298


## Which state had the highest covid cases?

In [ ]:
cases_unpivot.max()

## Which state had the most covid cases?

In [ ]:
cases_unpivot.groupby('state').sum()